In [10]:
from pyspark.sql import *
from pyspark.sql.types import *
from time import time
from functools import reduce
from GetPath import *
import pandas as pd
import shutil

In [11]:
# Config
input_path='D:/Output/chenbingying/A1/'
output_path='D:/Output/chenbingying/A2.csv'
place_path='D:/Code/chenbingying/Data/Place.csv'

In [12]:
# 区匹配
def Match_County(x,y):
    if(y is not None):
        if(y.find('广州市')!=-1):
            y=y.split("广州市")[1]
            for i in [['芳村区','荔湾区'],['花县','花都区'],['东山区','越秀区'],['萝岗区','黄埔区']]: 
                y=y.replace(i[0],i[1])
            return y
    if(x is not None):
        for i in [['芳村区','荔湾区'],['花县','花都区'],['东山区','越秀区'],['萝岗区','黄埔区']]:
            x=x.replace(i[0],i[1])   
        for i in ['荔湾','越秀','海珠','天河','白云','黄埔','番禺','花都','南沙','从化','增城']:
            if i in str(x):
                return i+'区'
        x1=['沙面','岭南','华林','多宝','昌华','逢源','龙津','金花','彩虹','南源','西村','站前','桥中','白鹤洞','冲口','花地','石围塘','茶滘','东漖','海龙','东沙','中南','洪桥','北京','六榕','流花','光塔','人民','东山','农林','梅花村','黄花岗','华乐','建设','大塘','珠光','大东','白云','登峰','矿泉','赤岗','新港','昌岗','江南中','滨江','素社','海幢','南华西','龙凤','沙园','南石头','凤阳','瑞宝','江海','琶洲','南洲','华洲','官洲','五山','员村','车陂','沙河','石牌','沙东','天河南','林和','兴华','棠下','天园','猎德','冼村','元岗','黄村','长兴','龙洞','凤凰','前进','珠吉','新塘','三元里','松洲','景泰','同德','黄石','棠景','新市','同和','京溪','永平','嘉禾','均禾','石井','金沙','云城','鹤龙','白云湖','石门','大源','龙归','人和','太和','钟落潭','江高','黄埔','红山','鱼珠','大沙','文冲','穗东','南岗','长洲','夏港','萝岗','云埔','联和','永和','长岭','九佛','龙湖','新龙','市桥','沙头','东环','桥南','小谷围','大石','洛浦','石壁','钟村','大龙','南村','新造','化龙','石楼','沙湾','石基','新华','花城','秀全','新雅','梯面','花山','花东','炭步','赤坭','狮岭','南沙','珠江','龙穴','万顷沙','横沥','黄阁','东涌','大岗','榄核','街口','江埔','城郊','温泉','良口','吕田','太平','鳌头','黄龙带水库','大岭山林场','荔城','增江','朱村','永宁','荔湖','宁西','新塘','石滩','中新','正果','派潭','小楼','仙村','荔联','雅瑶','东区','九龙','流溪河林场','黄龙带水库管理处']
        x2=['荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','荔湾','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','越秀','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','海珠','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','天河','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','白云','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','黄埔','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','番禺','花都','花都','花都','花都','花都','花都','花都','花都','花都','花都','南沙','南沙','南沙','南沙','南沙','南沙','南沙','南沙','南沙','从化','从化','从化','从化','从化','从化','从化','从化','从化','从化','增城','增城','增城','增城','增城','增城','增城','增城','增城','增城','增城','增城','增城','黄埔','花都','黄埔','黄埔','从化','从化']
        for i in range(len(x1)):
            if(x1[i] in x):
                return x2[i]+'区'
    return None

In [13]:
# 是否广州匹配
def Is_GuangZhou(x):
    if(x is not None):
        for i in address:
            if i in x:
                return '广州市'
    return None

In [14]:
def Create_A2(input_path,output_path,place_path):
    t_start=time()
    spark= SparkSession.builder.appName('A2_Create').config('spark.dynamicAllocation.enabled','true').config('spark.debug.maxToStringFields', '100').config('spark.sql.execution.arrow.enable','true').config('spark.executor.memory','48g').config('spark.driver.memory', '48g').config('spark.core.connection.ack.wait.timeout','36000s').config('spark.executor.heartbeatInterval','36000s').config('spark.network.timeout', '50000s').config('spark.rpc.lookupTimeout', '5000s').config('spark.shuffle.io.connectionTimeout', '50000s').getOrCreate()
    # 读取字典
    address_df=pd.read_csv(place_path,header=0,encoding='GBK')
    global address
    address=[]
    for i in address_df.columns:
        address+=address_df[i].dropna().to_list()
    # 读入文件
    files=[]
    filelist=GetPath(input_path,'tb_patient')
    for file in filelist:
        start=time()
        print('开始处理：',file.split('/')[-1])
        files.append(spark.read.option('inferSchema','true').option('header','true').csv(file))
        print('读入数据耗时：%.2f秒'%(time()-start))
    # 合并数据
    start=time()
    reduce(lambda x,y:x.unionByName(y), files).createOrReplaceTempView('A2_Raw')
    # 注册函数
    spark.udf.register('Match_County',Match_County,StringType())
    spark.udf.register('Is_GuangZhou',Is_GuangZhou,StringType())
    # 匹配数据
    spark.sql('''
    select MEDICAL_INSTITUTION_CODE
    ,FILE_NUMBER
    ,ID_NUMBER
    ,CARD_NUMBER
    ,CARD_TYPE
    ,case when id_birth_date is not null then to_date(id_birth_date) else to_date(BIRTH_DATE) end as BIRTH_DATE
    ,case when id_gender is not null then id_gender else GENDER end as GENDER
    ,ADDRESS_OF_EMPLOYER
    ,concat_ws('',CITY_OF_RESIDENCE,COUNTY_OF_RESIDENCE,RESIDENCE_ADDRESS) as RESIDENCE_ADDRESS
    ,concat_ws('',CITY_OF_REG_RESIDENCE,COUNTY_OF_REG_RESIDENCE,REGISTERED_ADDRESS) as REGISTERED_ADDRESS
    ,AREA
    ,Is_GuangZhou(concat_ws('',ADDRESS_OF_EMPLOYER,CITY_OF_RESIDENCE,COUNTY_OF_RESIDENCE,RESIDENCE_ADDRESS,CITY_OF_REG_RESIDENCE,COUNTY_OF_REG_RESIDENCE,REGISTERED_ADDRESS,AREA)) as CITY
    ,Match_County(concat_ws('',ADDRESS_OF_EMPLOYER,CITY_OF_RESIDENCE,COUNTY_OF_RESIDENCE,RESIDENCE_ADDRESS,CITY_OF_REG_RESIDENCE,COUNTY_OF_REG_RESIDENCE,REGISTERED_ADDRESS),AREA) as DISTRICT
    from A2_Raw
    ''').coalesce(1).write.csv(output_path+'A2',mode='overwrite',header=True)
    # 释放资源
    spark.catalog.dropTempView('A2_Raw')
    shutil.move(GetPath(output_path+'A2/','csv'),output_path)
    shutil.rmtree(output_path+'A2')
    print('匹配完成，耗时：%.2f秒'%(time()-start))
    print(f'已输出至：{output_path}')
    print('总耗时：%.2f秒'%(time()-t_start))

In [15]:
if __name__=="__main__":
    Create_A2(input_path,output_path,place_path)

开始处理： tb_patient_information.csv
读入数据耗时：41.40秒
开始处理： tb_patient_information_201807.csv
读入数据耗时：79.57秒
匹配完成，耗时：12252.95秒
已输出至：D:/Output/zhuhao/A2.csv
总耗时：12373.99秒
